In [1]:
import os
os.chdir("/gpfs/home/asun/jin_lab/yap/pipeline0_bt2_local_alignment/split_s10")
print(os.getcwd())

/gpfs/group/jin/asun/yap/pipeline0_bt2_local_alignment/split_s10


In [2]:
import pandas as pd
import numpy as np
import subprocess
from Bio.Seq import Seq
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
from cemba_data.utilities import get_configuration
from cemba_data.mapping import cutoff_vs_cell_remain, plot_on_plate

In [5]:
def test_xlim(data, hue, color_quantile=color_quantile, config=config):
    vmin = data[hue].min()
    vmax = data[hue].max()
    print(vmin, vmax)
    fig2, plate_names, plate_datas = plot_on_plate(
        data=data,
        hue=hue,
        groupby=plate_col,
        vmin=0,
        vmax=vmax,
        aggregation_func=lambda i: i.mean())
    return

In [4]:
output_dir = ''
plate_col = 'Plate'
color_quantile = (0.025, 0.975)

# Parameters
output_dir = "/gpfs/group/jin/asun/yap/pipeline2_yap_s10/yap_demultiplex"
output_dir = pathlib.Path(output_dir)
mapping_summary = pd.read_csv(output_dir / 'stats/MappingSummary.csv.gz', index_col=0)
config = get_configuration(output_dir / 'mapping_config.ini')

In [6]:
# Function to reverse complement if RC column is "RC"
def conditional_reverse_complement(row):
    if row["RC"] == "RC":
        return str(Seq(row["SEQ"]).reverse_complement())
    return row["SEQ"]  # Keep the sequence unchanged if not "RC"

# Load BAM into DataFrame
sam_output = subprocess.run(["samtools", "view", "intersect.bam"], capture_output=True, text=True)
sam_lines = sam_output.stdout.strip().split("\n")

# Convert SAM to DataFrame
columns = ["QNAME", "FLAG", "RNAME", "POS", "MAPQ", "CIGAR", "RNEXT", "PNEXT", "TLEN", "SEQ", "QUAL"]
data = [line.split("\t")[:11] for line in sam_lines]
df = pd.DataFrame(data, columns=columns)

# Convert FLAG to numeric
df["FLAG"] = pd.to_numeric(df["FLAG"])
df["RC"] = df["FLAG"].apply(lambda x: "RC" if x & 16 else "FORWARD")

rp_indices = pd.read_csv("/gpfs/home/asun/jin_lab/yap/raw_data/384RPIndexes.csv")
rp_indices = rp_indices.rename(columns={"RP Index": "RP_INDEX", "Position": "WELL"})   

# Apply the function to create the TRUE_SEQ column
df["TRUE_SEQ"] = df.apply(conditional_reverse_complement, axis=1)

df["RP_INDEX"] = df["TRUE_SEQ"].apply(lambda x: x[:8])

# Assign read type
df["READ_TYPE"] = df["FLAG"].apply(lambda x: "READ_1" if x & 64 else ("READ_2" if x & 128 else "UNPAIRED"))

df["READ_1"] = ""

readnames = df.loc[(df["READ_TYPE"] == "READ_2"), "QNAME"].tolist()

with open("grna_r1.fq", "r") as fq:
    while True:
        header = fq.readline().strip()  
        seq = fq.readline().strip()     
        fq.readline()                   
        fq.readline()                   
        
        if not header:
            break  # End of file
        
        qname = header.lstrip("@")

        if qname in df["QNAME"].values:
            first_8_bp = seq[:8]  # Extract the first 8 bases
            df.loc[df["QNAME"] == qname, "READ_1"] = seq
            df.loc[df["QNAME"] == qname, "RP_INDEX"] = first_8_bp


df = df.merge(rp_indices, on=["RP_INDEX"], how="left")
df_filtered = df.drop_duplicates(subset='QNAME', keep='first')
len(df_filtered)

result = (
    df_filtered.groupby(['WELL', 'RNAME'])
    .size()
    .unstack(fill_value=0)
)

# This gives you the desired result
result = result.astype(int)
result = result.reset_index()

In [12]:
# Filtering condition
filtered_df = df[(df["RNEXT"] != "=") | (df["READ_TYPE"] == "READ_1")]
len(filtered_df)

99

In [ ]:
from IPython.display import display

# Temporarily set options within this context
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.max_colwidth', None,
                       'display.width', None):
    display(df)

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL,RC,TRUE_SEQ,RP_INDEX,READ_TYPE,READ_1,WELL
0,lh00134:653:22MKYCLT4:3:1102:23021:5428,177,Foxg1_g1,1,22,66S72M12S,chr8,30412233,0,TTACCGTAACTTGAAAGTATTTCGATTTCTTGGCTTTATATATCTTGTGGAAAGGACGAAACACCGAGCGCGTTGTAGCTGAACGGGTTTTAGAGCTAGAAATAGCAAATTAAAATAAAACTAATCCATTATCAACTTAATATTTCCCTC,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIII9IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII,RC,GAGGGAAATATTAAGTTGATAATGGATTAGTTTTATTTTAATTTGCTATTTCTAGCTCTAAAACCCGTTCAGCTACAACGCGCTCGGTGTTTCGTCCTTTCCACAAGATATATAAAGCCAAGAAATCGAAATACTTTCAAGTTACGGTAA,ACAACAGC,READ_2,ACAACAGCATTTTTTTTTTTTTTTTTTTTTTTTTTTTTTACATCTACGACAAAAGGTTGTTAGAGAGATACTTGGACTTCCTTTGCCTGCAAACACAAAGATCTTAGTACAAAATACGTGACGTAGAAAGTACCAATTTCTTGGGGTTTG,J17
1,lh00134:653:22MKYCLT4:3:1107:31103:16231,99,Foxg1_g2,1,9,27S82M41S,=,1,-180,TACTGCTCTCATTTCCCGGCCAATGCAGAGTTACAACGGGACCACGTGTTTAAGAGCTATGCTGGAAACAGCATAGCAAGTTTAAATAAAACTAATCCATTATCAACTTAATATCCTCCCAGATCGGAAGAGCACACGTCTGAACTCCAG,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII,FORWARD,TACTGCTCTCATTTCCCGGCCAATGCAGAGTTACAACGGGACCACGTGTTTAAGAGCTATGCTGGAAACAGCATAGCAAGTTTAAATAAAACTAATCCATTATCAACTTAATATCCTCCCAGATCGGAAGAGCACACGTCTGAACTCCAG,TACTGCTC,READ_1,TACTGCTCTCATTTCCCGGCCAATGCAGAGTTACAACGGGACCACGTGTTTAAGAGCTATGCTGGAAACAGCATAGCAAGTTTAAATAAAACTAATCCATTATCAACTTAATATCCTCCCAGATCGGAAGAGCACACGTCTGAACTCCAG,I1
3,lh00134:653:22MKYCLT4:3:1121:13337:8917,83,Foxg1_g1,1,9,60S73M17S,=,1,201,CCCAGATCTAGACACTCGTGACTGGAGTTCAGACGTGTGCTCTTCCGATCTGGTATGGGGAGCGCGTTGTAGCTGAACGGGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGATATGCGC,9II99IIIIIIIIIII9IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII,RC,GCGCATATCCACTTTTTCAAGTTGATAACGGACTAGCCTTATTTTAACTTGCTATTTCTAGCTCTAAAACCCGTTCAGCTACAACGCGCTCCCCATACCAGATCGGAAGAGCACACGTCTGAACTCCAGTCACGAGTGTCTAGATCTGGG,GCGCATAT,READ_1,GCGCATATCCACTTTTTCAAGTTGATAACGGACTAGCCTTATTTTAACTTGCTATTTCTAGCTCTAAAACCCGTTCAGCTACAACGCGCTCCCCATACCAGATCGGAAGAGCACACGTCTGAACTCCAGTCACGAGTGTCTAGATCTGGG,O23
5,lh00134:653:22MKYCLT4:3:1130:28635:15180,99,Foxg1_g1,1,22,40S44M66S,=,1,-211,ACAACAGCTTCATATATCTTGTGGAAAGGACGAAACACCGAGCGCGTTGTAGCTGAACGGGTTTTAGAGCTAGAGATAGCAAGTATCCCAGATCGGAAGAGCACACGTCTGAACTCCAGTCACGAGTGTCTAGATCTGGGGTGCCGTCTT,IIIIIII9IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII9IIIIIIIII***9******9,FORWARD,ACAACAGCTTCATATATCTTGTGGAAAGGACGAAACACCGAGCGCGTTGTAGCTGAACGGGTTTTAGAGCTAGAGATAGCAAGTATCCCAGATCGGAAGAGCACACGTCTGAACTCCAGTCACGAGTGTCTAGATCTGGGGTGCCGTCTT,ACAACAGC,READ_1,ACAACAGCTTCATATATCTTGTGGAAAGGACGAAACACCGAGCGCGTTGTAGCTGAACGGGTTTTAGAGCTAGAGATAGCAAGTATCCCAGATCGGAAGAGCACACGTCTGAACTCCAGTCACGAGTGTCTAGATCTGGGGTGCCGTCTT,J17
7,lh00134:653:22MKYCLT4:3:1132:8677:2359,83,Dnmt3a_g1,3,9,31S71M48S,=,3,170,GACGTGTGCTCTTCCGATCTGGGAGGAAGGATAGAACTCAAAGAAGAGGGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGGTACTCTGCTTGATACCACATAGAACTCAAAGAAGAGGGGGAGCAGTA,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII*IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII,RC,TACTGCTCCCCCTCTTCTTTGAGTTCTATGTGGTATCAAGCAGAGTACCAAGTTGATAACGGACTAGCCTTATTTTAACTTGCTATTTCTAGCTCTAAAACCCTCTTCTTTGAGTTCTATCCTTCCTCCCAGATCGGAAGAGCACACGTC,TACTGCTC,READ_1,TACTGCTCCCCCTCTTCTTTGAGTTCTATGTGGTATCAAGCAGAGTACCAAGTTGATAACGGACTAGCCTTATTTTAACTTGCTATTTCTAGCTCTAAAACCCTCTTCTTTGAGTTCTATCCTTCCTCCCAGATCGGAAGAGCACACGTC,I1
9,lh00134:653:22MKYCLT4:3:1133:3257:17408,83,Foxg1_g1,1,22,102S38M10S,=,1,230,CCCCCCTTTTTTCAAGAAAAATATGCCATCCCAGATCTAGACACTCGTGACTGGAGTTCAGACGTGTGCTCTTCCGATCTGGAGGATGAATACTGCCGGGGGAGCGCGTTGTAGCTGAACGGGTTTTAGAGCTAGAAATATGGAGCAGTA,9999***99*9**9***9*99*****99**III9IIIII9IIIIII9IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

In [11]:
df

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL,RC,TRUE_SEQ,RP_INDEX,READ_TYPE,READ_1,WELL
0,lh00134:653:22MKYCLT4:3:1102:23021:5428,177,Foxg1_g1,1,22,66S72M12S,chr8,30412233,0,TTACCGTAACTTGAAAGTATTTCGATTTCTTGGCTTTATATATCTT...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIII9IIIIIIIIIIIIII...,RC,GAGGGAAATATTAAGTTGATAATGGATTAGTTTTATTTTAATTTGC...,ACAACAGC,READ_2,ACAACAGCATTTTTTTTTTTTTTTTTTTTTTTTTTTTTTACATCTA...,J17
1,lh00134:653:22MKYCLT4:3:1107:31103:16231,99,Foxg1_g2,1,9,27S82M41S,=,1,-180,TACTGCTCTCATTTCCCGGCCAATGCAGAGTTACAACGGGACCACG...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,FORWARD,TACTGCTCTCATTTCCCGGCCAATGCAGAGTTACAACGGGACCACG...,TACTGCTC,READ_1,TACTGCTCTCATTTCCCGGCCAATGCAGAGTTACAACGGGACCACG...,I1
2,lh00134:653:22MKYCLT4:3:1107:31103:16231,147,Foxg1_g2,1,9,57S82M11S,=,1,180,CTCTTTCCCTACACGACGCTCTTCCGATCTTACTGCTCTCATTTCC...,IIIIIIII9*9IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,RC,GGGAGGATATTAAGTTGATAATGGATTAGTTTTATTTAAACTTGCT...,TACTGCTC,READ_2,TACTGCTCTCATTTCCCGGCCAATGCAGAGTTACAACGGGACCACG...,I1
3,lh00134:653:22MKYCLT4:3:1121:13337:8917,83,Foxg1_g1,1,9,60S73M17S,=,1,201,CCCAGATCTAGACACTCGTGACTGGAGTTCAGACGTGTGCTCTTCC...,9II99IIIIIIIIIII9IIIIIIIIIIIIIIIIIIIIIIIIIIIII...,RC,GCGCATATCCACTTTTTCAAGTTGATAACGGACTAGCCTTATTTTA...,GCGCATAT,READ_1,GCGCATATCCACTTTTTCAAGTTGATAACGGACTAGCCTTATTTTA...,O23
4,lh00134:653:22MKYCLT4:3:1121:13337:8917,163,Foxg1_g1,1,9,9S73M68S,=,1,-201,GGTATGGGGAGCGCGTTGTAGCTGAACGGGTTTTAGAGCTAGAAAT...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,FORWARD,GGTATGGGGAGCGCGTTGTAGCTGAACGGGTTTTAGAGCTAGAAAT...,GCGCATAT,READ_2,GCGCATATCCACTTTTTCAAGTTGATAACGGACTAGCCTTATTTTA...,O23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,lh00134:653:22MKYCLT4:3:2473:7707:26236,153,Foxg1_g1,1,9,74S52M24S,=,1,0,ATCATATGCTTACCGTAACTTGAAAGTATTTCGATTTCTAGGCTTT...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII9IIIIIIIIIII...,RC,GGGGAAGAGGTTAATGGATTAGTTTTATTTTAACTTGCTATTTCTA...,ACAACAGC,READ_2,ACAACAGCACTTTTTTTTTTTTTTTTTTTTTTTTTTTTTCCCCCCC...,J17
190,lh00134:653:22MKYCLT4:3:2476:29622:10528,83,Foxg1_g1,1,9,53S73M24S,=,1,172,CAGACGTGTGCTCTTCCGATCTGAGGAATACTTGTGGAAAGGACGA...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,RC,ACAACAGCTATCAACCCAGAGTACCAAGTTGATAACGGACTAGCCT...,ACAACAGC,READ_1,ACAACAGCTATCAACCCAGAGTACCAAGTTGATAACGGACTAGCCT...,J17
191,lh00134:653:22MKYCLT4:3:2476:29622:10528,163,Foxg1_g1,1,9,31S73M46S,=,1,-172,GAGGAATACTTGTGGAAAGGACGAAACACCGAGCGCGTTGTAGCTG...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,FORWARD,GAGGAATACTTGTGGAAAGGACGAAACACCGAGCGCGTTGTAGCTG...,ACAACAGC,READ_2,ACAACAGCTATCAACCCAGAGTACCAAGTTGATAACGGACTAGCCT...,J17
192,lh00134:653:22MKYCLT4:3:2484:35018:20351,83,Foxg1_g1,1,9,17S73M60S,=,1,163,GCTCTTCCGATCTGGGGAGCGCGTTGTAGCTGAACGGGTTTTAGAG...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...,RC,GCGCATATTAACCATTCAGCTACAACGCGCTCCCCAAGCAGTGGTA...,GCGCATAT,READ_1,GCGCATATTAACCATTCAGCTACAACGCGCTCCCCAAGCAGTGGTA...,O23


In [ ]:
merged_local_mapping = pd.merge(mapping_summary, result, how="left", left_on="RandomIndex", right_on="WELL")
#merged_local_mapping.index = mapping_summary.index
merged_local_mapping = merged_local_mapping.drop(columns='WELL')
#merged_local_mapping = merged_local_mapping.fillna(0)

merged_local_mapping[result.columns[1:]] = merged_local_mapping[result.columns[1:]].fillna(0).astype(int)
merged_local_mapping["Total_grna"] = merged_local_mapping[result.columns[1:]].sum(axis=1)
merged_local_mapping[result.columns[1:]] = merged_local_mapping[result.columns[1:]].replace(0, np.nan)
merged_local_mapping["Total_grna"] = merged_local_mapping["Total_grna"].replace(0, np.nan)
merged_local_mapping